# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys_NoelleWatson import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("Starter_Code/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aasiaat,68.7098,-52.8699,1.19,97,100,2.04,GL,1695928462
1,1,wailua homesteads,22.0669,-159.3780,27.24,74,75,4.63,US,1695928496
2,2,avarua,-21.2078,-159.7750,25.03,78,40,10.29,CK,1695928437
3,3,opuwo,-18.0607,13.8400,22.98,40,0,3.87,NaN,1695928855
4,4,papatowai,-46.5619,169.4708,8.90,75,62,7.52,NZ,1695928438


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.55,
    hover_cols = ["City", "Humidity"]
)

# Display the map 
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 26 degrees celcius but higher than 60 degrees celcius.
filtered_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 26) & (city_data_df["Max Temp"] > 15)]

# Humidity less than 30%
filtered_city_data_df = filtered_city_data_df[filtered_city_data_df["Humidity"] < 40]

# Wind speed less than 8 mph.
filtered_city_data_df = filtered_city_data_df.loc[filtered_city_data_df["Wind Speed"] < 8]

# Cloudiness less than 15%.
filtered_city_data_df = filtered_city_data_df.loc[filtered_city_data_df["Cloudiness"] < 20]

# Drop any rows with null values
filtered_city_data_df = filtered_city_data_df.dropna()

# Display sample data
filtered_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,weyburn,49.6668,-103.8511,19.12,30,0,6.17,CA,1695928857
17,17,kapuskasing,49.4169,-82.4331,21.05,32,0,3.60,CA,1695928548
52,52,ghadamis,30.1337,9.5007,25.79,33,0,4.14,LY,1695928550
91,91,vegreville,53.5001,-112.0518,17.39,39,10,1.54,CA,1695928879
117,117,azadshahr,34.7909,48.5707,15.27,25,0,1.03,IR,1695928885


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df.copy()
hotel_df = hotel_df.drop(["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"], axis=1)
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
9,weyburn,CA,49.6668,-103.8511,30,
17,kapuskasing,CA,49.4169,-82.4331,32,
52,ghadamis,LY,30.1337,9.5007,33,
91,vegreville,CA,53.5001,-112.0518,39,
117,azadshahr,IR,34.7909,48.5707,25,
128,mount isa,AU,-20.7333,139.5000,30,
129,chibougamau,CA,49.9168,-74.3659,37,
193,sault ste. marie,CA,46.5168,-84.3333,37,
228,kandahar,AF,31.0000,65.7500,23,
250,leningradskaya,RU,46.3214,39.3877,31,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["address_line1"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
weyburn - nearest hotel: No hotel found
kapuskasing - nearest hotel: Travelodge
ghadamis - nearest hotel: Ein Al Fares Hotel
vegreville - nearest hotel: Hotello
azadshahr - nearest hotel: هتل آموزش
mount isa - nearest hotel: Ibis Styles
chibougamau - nearest hotel: No hotel found
sault ste. marie - nearest hotel: Holiday Inn
kandahar - nearest hotel: No hotel found
leningradskaya - nearest hotel: Старая мельница
qarah zia' od din - nearest hotel: No hotel found
chayan - nearest hotel: No hotel found
wainwright - nearest hotel: Park Hotel
ghanzi - nearest hotel: Ghanzi farmhouse
khuzdar - nearest hotel: Al-Mustafa hotel
khamis mushait - nearest hotel: No hotel found
zahedan - nearest hotel: hotel saleh;Bank Melli Iran
pampierstad - nearest hotel: No hotel found
vallenar - nearest hotel: Hotel del Marqués
figuig (centre) - nearest hotel: Hassan II Boulevard


,City,Country,Lat,Lng,Humidity,Hotel Name
9,weyburn,CA,49.6668,-103.8511,30,No hotel found
17,kapuskasing,CA,49.4169,-82.4331,32,Travelodge
52,ghadamis,LY,30.1337,9.5007,33,Ein Al Fares Hotel
91,vegreville,CA,53.5001,-112.0518,39,Hotello
117,azadshahr,IR,34.7909,48.5707,25,هتل آموزش
128,mount isa,AU,-20.7333,139.5000,30,Ibis Styles
129,chibougamau,CA,49.9168,-74.3659,37,No hotel found
193,sault ste. marie,CA,46.5168,-84.3333,37,Holiday Inn
228,kandahar,AF,31.0000,65.7500,23,No hotel found
250,leningradskaya,RU,46.3214,39.3877,31,Старая мельница


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=0.55,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)